In [21]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
from bs4 import BeautifulSoup
import pandas as pd
import os


def login_instacart(home_page): #locate, and enter the login information
    driver.get(home_page)
    time.sleep(15)
    email = driver.find_element('xpath',"//input[contains(@type,'email')]")
    email.send_keys(#Enter Acct User Name as String)
    password = driver.find_element('xpath', "//input[contains(@type,'password')]")
    password.send_keys(#Enter Acct Password as String)
    time.sleep(15)
    login = driver.find_element('xpath',"//button[contains(@class,'e-ztomkz')]")
    login.click()
    time.sleep(15)

def scroll_bottom_of_page():#load the entire web page by scrolling to the bottom
    pause_time_to_load = 30    
    last_height = driver.execute_script("return document.body.scrollHeight")
    while True:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")#scroll to bottom
        time.sleep(pause_time_to_load)
        try:  #remove hidden element that prevents from clicking on load more button and scrolling to bottom
            driver.find_element('xpath',"//div[contains(@class,'e-1wf5ba')]").remove()
        except:
            pass
        try: #click load more button if it is there
            Load_More_Button = driver.find_element('xpath',"//span[contains(@class,'e-15utg5h')]")
            Load_More_Button.click()
        except:
            pass 
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height: #if you can't scroll anymore break the loop else keep scrolling
            time.sleep(20)
            break
        last_height = new_height
        
def find_prices(soup): #load the entire web page in bs4, and locate the prices
    prices = []
    for price in soup.find_all('div', class_ = 'e-13udsys'):
        try:
            prices.append(price.find('div', class_ = 'e-k008qs')["aria-label"])
        except:
            prices.append(None) 
    return prices

def find_titles(soup): #load the entire web page in bs4, and locate the titles
    titles = []
    for title in soup.find_all('div', class_ = 'e-13udsys'):
        try:
            titles.append(title.find('span', class_ = 'e-th3ch3').contents[0])
        except:
            titles.append(None) 
    return titles

def find_quantitys(soup): #load the entire web page in bs4, and locate the quantitys
    qauntitys = []
    for qauntity in soup.find_all('div', class_ = 'e-13udsys'):
        try:
            qauntitys.append(qauntity.find('div', class_ = 'e-k9ly30')['title'])
        except:
            qauntitys.append(None)
    return qauntitys

def store_data(item,store,prices,titles,qauntitys): #store data into an excel sheet
    mined_data = pd.DataFrame(
        {'Prices': prices,
            "Title": titles,
            'Quantities': qauntitys 
        }) #populate data frame with data
    store_directory = '/'+store.rsplit('/', 2)[1]
    store_col = []
    item_col = []
    for price in prices: #add two columns of extra detail
        store_col.append(store_directory[1:]) #fill an entire column with the store being mined for instance make a column that just says "walmart"
        item_col.append(item) #fill an entire column with the item search being mined for instance make a column that just says "seafood"
    mined_data["Store"] = store_col
    mined_data["Item"] = item_col 
    store_directory = '/'+store.rsplit('/', 2)[1]
    if os.path.exists(os.getcwd()+store_directory) == False:
        os.makedirs(os.getcwd()+store_directory)
    final_destination = os.getcwd()+store_directory
    mined_data.to_excel(final_destination+'/'+item+'.xlsx') #store the data frame in excel
    
home_page = 'https://www.instacart.com/store/?categoryFilter=homeTabForYou'

stores = ['https://www.instacart.com/store/big-y/storefront', 
          'https://www.instacart.com/store/stop-shop/storefront', 
          'https://www.instacart.com/store/shoprite/storefront',
          'https://www.instacart.com/store/target/storefront']

search_keys = ['Fruits','Vegetables','Canned Goods','Dairy',
         'Meat','Fish & Seafood','Deli','Condiments & Spices','Snacks',
         'Bread & Bakery','Beverages','Pasta, Rice & Cereal','Baking','Frozen Foods']

#load driver, and log into instacart
options = webdriver.FirefoxOptions()
options.add_argument('--ignore-certificate-errors')
options.add_argument('--private')
options.add_argument('--headless')
driver = webdriver.Firefox(options)
driver.maximize_window()
login_instacart(home_page)
#repeat the scraping process for all the stores in the list of stores
for store in stores: 
    driver.get(store)
    time.sleep(20)
    #scrape by searching for an item, loading the entire page, and retrieving all the data on that page
    for Item in search_keys:
        time.sleep(20)
        search_bar = driver.find_element('xpath',"//input[contains(@class,'e-13nun8m')]")
        time.sleep(20)
        search_bar.send_keys(Item + Keys.ENTER)
        time.sleep(20)
        scroll_bottom_of_page()
        page_source = driver.page_source
        soup = BeautifulSoup(page_source,'lxml')
        Prices = find_prices(soup)
        time.sleep(5)
        Qauntitys = find_quantitys(soup)
        time.sleep(5)
        Titles = find_titles(soup)
        store_data(Item,store,Prices,Titles,Qauntitys)
        time.sleep(5)
        driver.back()
    
driver.quit()

In [4]:
import os
import pandas as pd

#locate all the store folders, then combine all the indidivual excel sheets into one master sheet for each store
directory_path = os.getcwd()
if os.path.exists(directory_path) and os.path.isdir(directory_path):
    folder_paths = []
    for root, dirs, files in os.walk(directory_path):
        for folder in dirs:
            folder_paths.append(os.path.join(root, folder))
    #for all the files in the current directory, only open the store directorys
    for folder in folder_paths:
        if folder.rsplit('\\', 1)[1] == '.ipynb_checkpoints':
            pass
        else:
            excel_directory = folder
            dataframes = []
            #locate all the files in the store directory, and only open excel files into a pandas data frame
            for filename in os.listdir(excel_directory):
                if filename.endswith('.xlsx') or filename.endswith('.xls'):  
                    file_path = os.path.join(excel_directory, filename)
                    df = pd.read_excel(file_path) 
                    dataframes.append(df)
            #combine all the data frames into one store master frame
            combined_dataframe = pd.concat(dataframes, ignore_index=True)           
            combined_dataframe = combined_dataframe.drop(['Unnamed: 0'], axis = 1)
            combined_dataframe = combined_dataframe.dropna(subset=['Prices'])
            combined_dataframe.to_excel(folder+'\\'+folder.rsplit('\\')[-1]+'_All_Data.xlsx')